# SSL, Fine Tuning, and Linear Probing Heads

> none of these work! 

In [ ]:
#| default_exp heads

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch, torch.nn.functional as F

from torch import nn
from torch.nn import init
from pftsleep.transformers import TSTEncoderLayer
from pftsleep.layers import Transpose, get_activation_fn, EmbeddingAugmentations
from pftsleep.augmentations import create_patch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

## Linear Probing and Fine Tuning Heads

In [ ]:
#| export
class RNNProbingHead(nn.Module):
    def __init__(self, 
                 c_in, 
                 input_size, 
                 hidden_size, 
                 n_classes, 
                 contrastive=False, 
                 module='GRU', 
                 linear_dropout=0., 
                 rnn_dropout=0., 
                 num_rnn_layers=1, 
                 act='gelu', 
                 pool='average', # 'average' or 'max' or 'majority'
                 temperature=2., # only used if pool='majority'
                 n_linear_layers=1, 
                 predict_every_n_patches=1, 
                 bidirectional=True, 
                 affine=False, 
                 shared_embedding=True,
                 augmentations=None,
                 augmentation_mask_ratio=0.,
                 augmentation_dims_to_shuffle=[1,2,3],
                 norm=None # one of [None, 'pre', 'post']
                 ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.c_in = c_in
        self.rnn_dropout = rnn_dropout
        self.linear_dropout = linear_dropout
        self.bidirectional = bidirectional
        self.n_classes = n_classes
        self.n_linear_layers = n_linear_layers
        self.act = act
        self.pool = pool
        self.num_rnn_layers = num_rnn_layers
        self.predict_every_n_patches = predict_every_n_patches
        self.affine = affine
        self.shared_embedding = shared_embedding
        self.contrastive = contrastive
        self.augmentations = augmentations
        self.temperature = temperature

        if augmentations is not None:
            self.augmentations = augmentations
            self.augmentation_layer = EmbeddingAugmentations(augmentations=augmentations, dims_to_shuffle=augmentation_dims_to_shuffle, patch_mask_ratio=augmentation_mask_ratio, jitter_zero_mask_ratio=augmentation_mask_ratio)
        if norm in ['pre', 'post']:
            self.do_norm = norm
            self.norm_layer = nn.BatchNorm2d(self.c_in)# nn.LayerNorm(self.hidden_size * (2 if self.bidirectional else 1))
        else:
            self.do_norm = False
            self.norm_layer = nn.Identity()
        self.flatten = nn.Flatten(start_dim=-3, end_dim=-2)
        #self.attn = MultiheadAttentionCustom(d_model=input_size, n_heads=n_heads)

        if self.affine:
            self.affine_weight = nn.Parameter(torch.ones(1,c_in,1))
            self.affine_bias = nn.Parameter(torch.zeros(1,c_in,1))

        if module.lower() == 'gru':
            self.rnn = nn.GRU(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=num_rnn_layers, bias=True, batch_first=True, dropout=rnn_dropout, bidirectional=self.bidirectional)
            if self.contrastive:
                self.rnn2 = nn.GRU(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=num_rnn_layers, bias=True, batch_first=True, dropout=rnn_dropout, bidirectional=self.bidirectional)
        else:
            self.rnn = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=num_rnn_layers, bias=True, batch_first=True, dropout=rnn_dropout, bidirectional=self.bidirectional)
            if self.contrastive:
                self.rnn2 = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=num_rnn_layers, bias=True, batch_first=True, dropout=rnn_dropout, bidirectional=self.bidirectional)
        self.layers = []
        #n_nuerons = int(((hidden_size*2 if self.bidirectional else hidden_size) - n_classes)/self.n_linear_layers) if self.n_linear_layers >= 1 else 0
        # if self.n_linear_layers == 1:
        #     self.layers.append(nn.Sequential(nn.Linear(hidden_size*2 if self.bidirectional else hidden_size, n_classes)))
        # else:
        #     for i in range(self.n_linear_layers - 1):
        #         self.layers.append(nn.Sequential(nn.Linear(n_nuerons*i+(hidden_size*2 if self.bidirectional else hidden_size), (hidden_size*2 if self.bidirectional else hidden_size)+n_nuerons*(i+1)), nn.BatchNorm2d(c_in), get_activation_fn(act), nn.Dropout(linear_dropout)))
        #     self.layers.append(nn.Sequential(nn.Linear((hidden_size*2 if self.bidirectional else hidden_size)+n_nuerons*(i+1), n_classes)))
        # self.layers = nn.Sequential(*self.layers)
        f = 2 if self.bidirectional else 1
        f = f * 2 if self.contrastive else f
        if not self.shared_embedding:
            for _ in range(self.c_in): self.layers.append(nn.Linear(hidden_size*f, n_classes))
        else:
            self.layers.append(nn.Linear(hidden_size*f, n_classes)) # [bs x c_in x n_patches x n_classes]
        self.layers = nn.Sequential(*self.layers)
        #self.fc = nn.Linear(hidden_size*2 if self.bidirectional else hidden_size, n_classes)
        self.pool_kernel_size = int(self.c_in*self.predict_every_n_patches) if self.predict_every_n_patches > 1 else self.c_in

        if pool in ['average', 'max']:
            self.pool_layer = nn.AvgPool1d(kernel_size=self.pool_kernel_size, stride=self.pool_kernel_size) if self.pool == 'average' else nn.MaxPool1d(kernel_size=self.pool_kernel_size, stride=self.pool_kernel_size)
        elif pool == 'majority':
            self.pool_layer = nn.Softmax(dim=-1)
        else:
            raise ValueError(f"Pooling type {pool} not supported. Choose one of ['average', 'max', 'majority']")
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, return_softmax=False, sequence_padding_mask=None):
        """
        GRU input expects [bs x seq_len x H_in]
        x: [bs x n channels x d model x n patches]
        """
        # pass each channel through GRU and concat
        bs, n_channels, d_model, n_patches = x.shape
        #x = x.permute(0,1,3,2) #[bs x n channels x n patches x d model]
        #x = x.reshape(-1, n_patches, d_model)
        #x, _ = self.attn(x)
        #x = x.reshape(bs, n_channels, n_patches, d_model).permute(0,2,1,3) # with just rnn only need permute(0, 3, 1, 2) [bs x n patches x n channels x d model]
        if self.augmentations is not None:
            x = self.augmentation_layer(x)
        if self.contrastive:
            x1 = x[:,:,:(d_model//2),:].clone()
            x2 = x[:,:,(d_model//2):,:].clone()

            x1, x2 = x1.permute(0, 3, 1, 2), x2.permute(0, 3, 1, 2)
            x1, x2 = self.flatten(x1), self.flatten(x2)
            x1,_ = self.rnn(x1)
            x2,_ = self.rnn2(x2)

            x1, x2 = torch.reshape(x1, (bs, n_patches, n_channels, -1)), torch.reshape(x2, (bs, n_patches, n_channels, -1))
            x_cat = torch.cat([x1,x2], dim=-1)
        else:
            if self.do_norm == 'pre':
                x = self.norm_layer(x) # [bs x n channels x d model x n patches]
            x_cat = x.permute(0, 3, 1, 2) # [bs x n patches x n channels x d model] 
            x_cat = self.flatten(x_cat) # [bs x n patches * n channels x d model] 
            x_cat,_ = self.rnn(x_cat) # [bs x n patches * n channels x hidden_size]
            x_cat = torch.reshape(x_cat, (bs, n_patches, n_channels, -1)) # [bs x n patches x n channels x hidden_size]
            if self.do_norm == 'post':
                x_cat = x_cat.permute(0, 2, 3, 1)  # [bs, n_channels, hidden_size, n_patches]
                x_cat = self.norm_layer(x_cat)
                x_cat = x_cat.permute(0, 3, 1, 2)  # [bs, n_patches, n_channels, hidden_size]
            
        if not self.shared_embedding:
            x_out = []
            for i in range(self.c_in):
                z_ = self.layers[i](x_cat[:,:,i,:])
                x_out.append(z_)
            x_cat = torch.stack(x_out, dim=2) # [bs x n patches x n channels x n classes]
        else:
            x_cat = self.layers(x_cat) # [bs x n patches x n channels x n classes]
        
        if self.affine:
            x_cat = x_cat * self.affine_weight
            x_cat = x_cat + self.affine_bias
        x_cat = self.flatten(x_cat) # [bs x n patches * n channels x n classes]
        if self.pool == 'majority':
            # Group patches that belong to same segment (30 seconds)
            n_segments = (n_patches * n_channels) // self.pool_kernel_size
            x_segments = x_cat.view(bs, n_segments, self.pool_kernel_size, self.n_classes) # [bs x n_segments x patches_per_segment x n_classes]
            # vote with a temperature scaling
            soft_votes = self.pool_layer(x_segments / self.temperature) # [bs x n_segments x patches_per_segment x n_classes]
            # majority vote!
            segment_decisions = soft_votes.sum(dim=2) # [bs x n_segments x n_classes]
            segment_decisions = segment_decisions.transpose(1,2) # [bs x n_classes x n_segments]
        else:
            x_cat = x_cat.transpose(1,2)
            segment_decisions = self.pool_layer(x_cat)
        #x = x.unfold(dimension=-1, size=self.avg_pool_kernel_stride, step=self.avg_pool_kernel_stride).median(dim=-1)[0]
        #x = x.transpose(1,2)
        #x = x.reshape(x.shape[0], self.c_in, -1, self.n_classes)
        if return_softmax:
            segment_decisions = self.softmax(segment_decisions)
        return segment_decisions
     


In [ ]:
#| export
class RNNProbingHeadExperimental(nn.Module):
    def __init__(self, 
                 c_in, 
                 input_size, 
                 hidden_size, 
                 n_classes, 
                 contrastive=False, # deprecated
                 module='GRU', 
                 linear_dropout=0., 
                 rnn_dropout=0., 
                 num_rnn_layers=1, 
                 act='gelu', 
                 pool='average', # 'average' or 'max' or 'majority'
                 temperature=2., # only used if pool='majority'
                 predict_every_n_patches=1, 
                 bidirectional=True, 
                 affine=False, 
                 augmentations=None,
                 augmentation_mask_ratio=0.,
                 augmentation_dims_to_shuffle=[1,2,3],
                 pre_norm=True, # one of [None, 'pre', 'post']
                 mlp_final_head=False
                 ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.c_in = c_in
        self.rnn_dropout = rnn_dropout
        self.linear_dropout = linear_dropout
        self.bidirectional = bidirectional
        self.n_classes = n_classes

        self.pool = pool
        self.num_rnn_layers = num_rnn_layers
        self.predict_every_n_patches = predict_every_n_patches
        self.affine = affine
        self.augmentations = augmentations
        self.temperature = temperature
        self.pre_norm = pre_norm

        if augmentations is not None:
            self.augmentations = augmentations
            self.augmentation_layer = EmbeddingAugmentations(augmentations=augmentations, dims_to_shuffle=augmentation_dims_to_shuffle, patch_mask_ratio=augmentation_mask_ratio, jitter_zero_mask_ratio=augmentation_mask_ratio)
        if pre_norm:
            self.norm_layer = nn.BatchNorm2d(self.c_in)# nn.LayerNorm(self.hidden_size * (2 if self.bidirectional else 1))
        else:
            self.norm_layer = nn.Identity()
        self.flatten = nn.Flatten(start_dim=-2, end_dim=-1) # flatten d_model and n channels

        if self.affine:
            self.affine_weight = nn.Parameter(torch.ones(1,c_in,1))
            self.affine_bias = nn.Parameter(torch.zeros(1,c_in,1))

        if module.lower() == 'gru':
            self.rnn = nn.GRU(input_size=self.input_size*self.c_in, hidden_size=self.hidden_size, num_layers=num_rnn_layers, bias=True, batch_first=True, dropout=rnn_dropout, bidirectional=self.bidirectional)
        else:
            self.rnn = nn.LSTM(input_size=self.input_size*self.c_in, hidden_size=self.hidden_size, num_layers=num_rnn_layers, bias=True, batch_first=True, dropout=rnn_dropout, bidirectional=self.bidirectional)
        self.layers = []
        
        f = 2 if self.bidirectional else 1
        
        if mlp_final_head:
            self.layers.append(nn.Sequential(
                nn.Linear(hidden_size*f, hidden_size*f//2),
                nn.LayerNorm([hidden_size*f//2]),
                nn.ReLU(),
                nn.Dropout(linear_dropout),
                nn.Linear(hidden_size*f//2, n_classes)
            ))
        else:
            self.layers.append(nn.Linear(hidden_size*f, n_classes)) # [bs x c_in x n_patches x n_classes]
        
        self.layers = nn.Sequential(*self.layers)
        #self.fc = nn.Linear(hidden_size*2 if self.bidirectional else hidden_size, n_classes)
        self.pool_kernel_size = int(self.predict_every_n_patches)

        if pool in ['average', 'max']:
            self.pool_layer = nn.AvgPool1d(kernel_size=self.pool_kernel_size, stride=self.pool_kernel_size) if self.pool == 'average' else nn.MaxPool1d(kernel_size=self.pool_kernel_size, stride=self.pool_kernel_size)
        elif pool == 'majority':
            self.pool_layer = nn.Softmax(dim=-1)
        else:
            raise ValueError(f"Pooling type {pool} not supported. Choose one of ['average', 'max', 'majority']")
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, return_softmax=False, sequence_padding_mask=None):
        """
        GRU input expects [bs x seq_len x H_in]
        x: [bs x n channels x d model x n patches]
        """
        # pass each channel through GRU and concat
        bs, n_channels, d_model, n_patches = x.shape
        if self.augmentations is not None:
            x = self.augmentation_layer(x)
        
        if self.pre_norm:
            x = self.norm_layer(x) # [bs x n channels x d model x n patches]
        x_cat = x.permute(0, 3, 1, 2) # [bs x n patches x n channels x d model] 
        
        if self.affine:
            x_cat = x_cat * self.affine_weight
            x_cat = x_cat + self.affine_bias
        
        x_cat = self.flatten(x_cat) # [bs x n patches x n channels * d model] 
        if sequence_padding_mask is not None:

            patch_len = sequence_padding_mask.shape[-1] // n_patches
            key_padding_mask = create_patch(sequence_padding_mask, patch_len=patch_len, stride=patch_len, constant_pad=True, constant_pad_value=1, max_seq_len=sequence_padding_mask.shape[-1])
            key_padding_mask = torch.all(key_padding_mask, -1).squeeze(-1).int() # [bs x n patches]
            # calc lengths of each sequence, 0s are valid inputs, 1s are padding
            lengths = n_patches - torch.flip(key_padding_mask, dims=[1]).argmin(dim=1) # find padding index at the end of each batch sample (only ignoring paddingh at the end of the sequence)
            #lengths = (key_padding_mask == 0).sum(dim=1).cpu()
            lengths = lengths.cpu()
            x_cat = pack_padded_sequence(x_cat, lengths, batch_first=True, enforce_sorted=False)
        x_cat,_ = self.rnn(x_cat) # [bs x n patches x hidden_size*f]

        if sequence_padding_mask is not None:
            x_cat, _ = pad_packed_sequence(x_cat, batch_first=True, total_length=n_patches)
        x_cat = self.layers(x_cat) # [bs x n patches x n classes]
        
        if self.pool == 'majority':
            # Group patches that belong to same segment (30 seconds)
            n_segments = n_patches // self.pool_kernel_size
            x_segments = x_cat.view(bs, n_segments, self.pool_kernel_size, self.n_classes) # [bs x n_segments x patches_per_segment x n_classes]
            # vote with a temperature scaling
            soft_votes = self.pool_layer(x_segments / self.temperature) # [bs x n_segments x patches_per_segment x n_classes]
            # majority vote!
            segment_decisions = soft_votes.sum(dim=2) # [bs x n_segments x n_classes]
            segment_decisions = segment_decisions.transpose(1,2) # [bs x n_classes x n_segments]
        else:
            x_cat = x_cat.transpose(1,2)
            segment_decisions = self.pool_layer(x_cat)
        #x = x.unfold(dimension=-1, size=self.avg_pool_kernel_stride, step=self.avg_pool_kernel_stride).median(dim=-1)[0]
        #x = x.transpose(1,2)
        #x = x.reshape(x.shape[0], self.c_in, -1, self.n_classes)
        if return_softmax:
            segment_decisions = self.softmax(segment_decisions)
        return segment_decisions
     


In [ ]:
#| notest
m = RNNProbingHeadExperimental(c_in=7, 
                                pool='average', 
                                input_size = 384, 
                                bidirectional=True,
                                affine=False, 
                                hidden_size=1200,
                                module='GRU',
                                n_classes=4,
                                predict_every_n_patches=32,
                                rnn_dropout=0.,
                                num_rnn_layers=1,
                                linear_dropout=0.,
                                mlp_final_head=False,
                                pre_norm=True)
x = torch.randn((4,7,384,960))
sequence_padding_mask = torch.zeros(4,960)
sequence_padding_mask[:,-32:] = 1
m(x, return_softmax=True, sequence_padding_mask=sequence_padding_mask).shape


torch.Size([4, 4, 30])

In [ ]:
#| notest
m = RNNProbingHead(c_in=7, pool='majority', input_size = 384, contrastive=False, bidirectional=True, affine=True, shared_embedding=False, hidden_size=384, module='GRU', n_classes=4, predict_every_n_patches=32, rnn_dropout=0., num_rnn_layers=1, linear_dropout=0., n_linear_layers=1, norm='post')
x = torch.randn((4,7,384,960))

m(x, return_softmax=True).shape

torch.Size([4, 4, 30])

In [ ]:
#| notest
m = RNNProbingHead(c_in=7, input_size = 512, contrastive=True, bidirectional=True, affine=False, shared_embedding=True, hidden_size=256, module='GRU', n_classes=5, predict_every_n_patches=5, rnn_dropout=0., num_rnn_layers=1, linear_dropout=0., n_linear_layers=1)
x = torch.randn((4,7,512*2,3600))

m(x, return_softmax=True).shape

torch.Size([4, 5, 720])

In [ ]:
#| export
class TransformerDecoderProbingHead(nn.Module):
    def __init__(self, 
                 c_in, 
                 d_model, 
                 n_classes, 
                 norm='BatchNorm', 
                 dropout=0., 
                 act='gelu', 
                 d_ff=2048, 
                 num_layers=1, 
                 n_heads=2, 
                 predict_every_n_patches=1, 
                 affine=False, 
                 shared_embedding=True
                 ):


        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.num_layers = num_layers
        self.d_ff = d_ff
        self.act = act
        self.c_in = c_in
        self.dropout = dropout
        self.n_classes = n_classes
        self.predict_every_n_patches = predict_every_n_patches
        self.affine = affine
        self.shared_embedding = shared_embedding

        if self.affine:
            self.affine_weight = nn.Parameter(torch.ones(1,c_in,1))
            self.affine_bias = nn.Parameter(torch.zeros(1,c_in,1))

        self.flatten = nn.Flatten(start_dim=-3, end_dim=-2)
        norm_m = nn.BatchNorm1d(d_model) if "batch" in norm.lower() else nn.LayerNorm([c_in, n_patches, d_model])
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=n_heads, dropout=dropout, dim_feedforward=d_ff, batch_first = True, activation=act)
        self.decoder =  nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        self.layers = []
        if not self.shared_embedding:
            for _ in range(self.c_in): self.layers.append(nn.Linear(d_model, n_classes))
        else:
            self.layers.append(nn.Linear(d_model, n_classes)) # [bs x c_in x n_patches x n_classes]
        self.layers = nn.Sequential(*self.layers)

        avg_pool_kernel_stride = int(self.c_in*self.predict_every_n_patches) if self.predict_every_n_patches > 1 else self.c_in
        self.avg_pool = nn.AvgPool1d(kernel_size=avg_pool_kernel_stride, stride=avg_pool_kernel_stride)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, return_softmax=False):
        """
        x: [bs x n channels x d model x n patches]
        """
        # pass each channel through GRU and concat
        bs, n_channels, d_model, n_patches = x.shape
        x = x.permute(0,1,3,2) #[bs x n channels x n patches x d model]
        x = x.reshape(-1, n_patches, d_model)
        x = self.decoder(x,x) # x is both target and memory
        x = x.reshape(bs, n_channels, n_patches, d_model).permute(0,2,1,3) # with just rnn only need permute(0, 3, 1, 2) [bs x n patches x n channels x d model]

        if not self.shared_embedding:
            x_out = []
            for i in range(self.c_in):
                z_ = self.layers[i](x[:,:,i,:])
                x_out.append(z_)
            x = torch.stack(x_out, dim=2) # [bs x n patches x n channels x n classes]
        else:
            x = self.layers(x) # [bs x n patches x n channels x n classes]
        
        if self.affine:
            x = x * self.affine_weight
            x = x + self.affine_bias

        x = self.flatten(x)
        x = x.transpose(1,2)
        x = self.avg_pool(x)
        #x = x.transpose(1,2)
        #x = x.reshape(x.shape[0], self.c_in, -1, self.n_classes)
        if return_softmax:
            x = self.softmax(x)
        return x

In [ ]:
#| notest
layer = TransformerDecoderProbingHead(c_in=7, affine=True, shared_embedding=False, d_model=512, n_classes=5, dropout=0., num_layers=1, n_heads=2, predict_every_n_patches=5)
x = torch.randn((4, 7, 512, 3600))

layer(x).shape

torch.Size([4, 5, 720])

In [ ]:
#| export
class DecoderFeedForward(nn.Module):
    """
    transformer decoder with attention for feedforward predictions. This is really just another encoder layer followed by
    a linear layer + 1d convolution + softmax. However, if used in linear probing, could be useful.
    """
    def __init__(self, 
                 c_in, # the number of input channels
                 predict_every_n_patches, # for a given sequence of length m with frequency f, number of predictions
                 num_layers,
                 d_ff,
                 attn_dropout,
                 res_attention, 
                 pre_norm, 
                 store_attn,
                 n_heads,
                 shared_embedding,
                 affine,
                 n_classes, # the number of classes to predict (for sleep stage - there are 6)
                 d_model, # the dimension of the transformer model
                 norm='BatchNorm', # batchnorm or layernorm between linear and convolutional layers
                 act='gelu', # activation function to use between layers, 'gelu' or 'relu'
                 dropout=0. # dropout in between linear layers
                 ):
        super().__init__()
        assert n_classes >= 2, "The number of classes must be >= 2. If you want to perform a binary task, pass in the value 2."
        self.c_in = c_in
        self.n_classes = n_classes
        self.predict_every_n_patches = predict_every_n_patches
        self.d_model = d_model
        self.num_layers = num_layers
        self.d_ff = d_ff
        self.attn_dropout = attn_dropout
        self.res_attention = res_attention
        self.pre_norm = pre_norm
        self.store_attn = store_attn
        self.n_heads = n_heads
        self.norm = norm
        self.act = act
        self.dropout = dropout
        self.shared_embedding = shared_embedding
        self.affine = affine

        if self.affine:
            self.affine_weight = nn.Parameter(torch.ones(1,c_in,1))
            self.affine_bias = nn.Parameter(torch.zeros(1,c_in,1))
        
        self.decoder_layers = nn.ModuleList([TSTEncoderLayer(d_model=d_model, n_heads=n_heads, d_ff=d_ff, norm=norm,
                                                      attn_dropout=attn_dropout, dropout=dropout,
                                                      activation=act, res_attention=res_attention,
                                                      pre_norm=pre_norm, store_attn=store_attn) for i in range(num_layers)])
        self.layers = []
        if not self.shared_embedding:
            for _ in range(self.c_in): self.layers.append(nn.Linear(d_model, n_classes))
        else:
            self.layers.append(nn.Linear(d_model, n_classes)) # [bs x c_in x n_patches x n_classes]
        self.layers = nn.Sequential(*self.layers)
        
        #self.linear_layer = nn.Sequential(nn.Linear(d_model, self.win_length), nn.BatchNorm2d(c_in) if "batch" in norm.lower() else nn.LayerNorm([c_in, n_patches, self.win_length]), get_activation_fn(act), nn.Dropout(dropout))
        #self.flatten = nn.Flatten(start_dim=-2) # now flatten everything to get full length sequences
        # now convolve with stride/kernel = frequency * n second predicitions
        #final_kernel_stride = int(frequency*predict_every_seconds) # this will make one predictions for every predict_every_seconds seq length
        #self.conv1d = nn.Conv1d(in_channels=c_in, out_channels=n_classes, kernel_size=final_kernel_stride, stride=final_kernel_stride)
        #self.unfold_size = int(self.predict_every_seconds/(self.win_length/self.frequency))
        self.flatten = nn.Flatten(start_dim=-3, end_dim=-2)
        self.avg_pool_kernel_stride = int(self.c_in*self.predict_every_n_patches) if self.predict_every_n_patches > 1 else self.c_in
        self.avg_pool = nn.AvgPool1d(kernel_size=self.avg_pool_kernel_stride, stride=self.avg_pool_kernel_stride)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x, key_padding_mask=None, return_softmax=False):
        """
        Make sure to freeze the encoder and remove its pretrain head! 

        in: [bs x n channels x d model x n patches]
        out: [bs x n classes x pred len]
        """
        bs, n_channels, d_model, n_patches = x.shape
        z = x.permute(0,1,3,2) #[bs x n channels x n patches x d model]
        z = z.reshape(-1, n_patches, d_model) # z: [bs * nvars x num_patch x d_model]
        if self.res_attention:
            scores = None
            for mod in self.decoder_layers:
                z, scores = mod(z, prev=scores, key_padding_mask=key_padding_mask) # z: [bs * n_vars x num_patch x d_model], scores: [bs * n_vars x n_heads x num_patch x num_patch]
        else:
            for mod in self.decoder_layers: 
                z = mod(z, key_padding_mask=key_padding_mask) # z: [bs * n_vars x num_patch x d_model]
        z = z.reshape(bs, n_channels, n_patches, d_model).permute(0,2,1,3) # z: [bs x n patches x n channels x d model]
        if not self.shared_embedding:
            x_out = []
            for i in range(self.c_in):
                z_ = self.layers[i](z[:,:,i,:])
                x_out.append(z_)
            z = torch.stack(x_out, dim=2) # [bs x n patches x n channels x n classes]
        else:
            z = self.layers(z) # [bs x n patches x n channels x n classes]

        if self.affine:
            z = z * self.affine_weight
            z = z + self.affine_bias
        z = self.flatten(z) # # [bs x n channels * n patches x n classes]
        z = z.transpose(1,2) # [bs x n classes x n channels * n patches]
        z  = self.avg_pool(z)
        #z = z.unfold(-2, size=self.unfold_size, step=self.unfold_size).sum(dim=-1) # take 30 seconds and sum up logits [bs x n channels x 30sec patches x n_classes]
        
        #z = self.flatten(z)
        #z=z.sum(dim=1) 
        #z = z.transpose(1,2)
        #z = self.conv1d(z)
        if return_softmax:
            z = self.softmax(z)
        return z

In [ ]:
#| notest
c_in = 7
frequency = 125
win_length=750 
overlap = 0.
hop_length=win_length - int(overlap*win_length)
max_seq_len_sec = (6*3600) # for dataloader
#seq_len_sec = sample_stride = 3*3600 # for dataloader
max_seq_len = max_seq_len_sec*frequency # for model
#n_patches = n_fft // 2 + 1
n_patches = (max(max_seq_len, win_length)-win_length) // hop_length + 1

#patch_len = int((win_length-conv_kernel_stride_size[1])/conv_kernel_stride_size[1] + 1)
x = torch.randn(2,c_in,512,n_patches)

model = DecoderFeedForward(c_in=c_in,
                           predict_every_n_patches=5,
                           num_layers=1,
                           d_ff = 2048,
                           attn_dropout=0.,
                           res_attention = False,
                           pre_norm = False,
                           store_attn = False,
                           n_heads=2,
                           affine=False,
                           shared_embedding=False,
                           n_classes=5,
                           d_model=512,
                           norm='BatchNorm',
                           act='gelu',
                           dropout=0.
                           )

model(x).shape

torch.Size([2, 5, 720])

In [ ]:
#| export
class TimeDistributedConvolutionalFeedForward(nn.Module):
    """
    Convolutional feed forward head that first uses a linear feed forward network to project features into the original convolutional dimension.
    Then, a convolutional transpose is used to extrapolate the data to its original form.
    Finally, a final convolution is used to predict the classes.
    """
    def __init__(self, 
                 c_in, # the number of input channels
                 frequency, # the frequency of the original channels
                 predict_every_seconds, # for a given sequence of length m with frequency f, number of predictions
                 n_classes, # the number of classes to predict (for sleep stage - there are 6)
                 win_length, # the convolved patch length, the first step in this is to do a linear layer to this dimension
                 d_model, # the dimension of the transformer model
                 affine = False,
                 shared_embedding=True
                 ):
        super().__init__()
        
        self.c_in = c_in
        self.n_classes = n_classes
        self.predict_every_seconds = predict_every_seconds
        self.d_model = d_model
        self.win_length = win_length
        self.frequency = frequency
        self.layers = []
        self.affine = affine
        self.shared_embedding = shared_embedding
        if self.affine:
            self.affine_weight = nn.Parameter(torch.ones(1,c_in,1))
            self.affine_bias = nn.Parameter(torch.zeros(1,c_in,1))

        if not self.shared_embedding:
            for _ in range(self.c_in): self.layers.append(nn.Linear(d_model, win_length))
        else:
            self.layers.append(nn.Linear(d_model, win_length)) # [bs x c_in x n_patches x n_classes]

        self.layers = nn.Sequential(*self.layers)
        # now we should probably to a 2d convtranspose to get to the original number of channels, now the channels and patch len are back to normal size
        self.flatten = nn.Flatten(start_dim=-2) # now flatten everything to get full length sequences
        # now convolve with stride/kernel = frequency * n second predicitions
        final_kernel_stride = int(frequency*predict_every_seconds) # this will make one predictions for every predict_every_seconds seq length
        self.conv1d = nn.Conv1d(in_channels=c_in, out_channels=n_classes, kernel_size=final_kernel_stride, stride=final_kernel_stride)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x, return_softmax=False):
        """
        Make sure to freeze the encoder and remove its pretrain head! 

        in: [bs x n channels x d model x n patches]
        out: [bs x n classes x pred len]
        """
        x = x.permute(0,3,1,2) # [bs x n patches x n channels x d model]
        if not self.shared_embedding:
            x_out = []
            for i in range(self.c_in):
                z_ = self.layers[i](x[:,:,i,:])
                x_out.append(z_)
            x = torch.stack(x_out, dim=2) # [bs x n patches x n channels x patch len]
        else:
            x = self.layers(x) # [bs x n patches x n channels x patch len]
        if self.affine:
            x = x * self.affine_weight
            x = x + self.affine_bias
        x = x.transpose(1,2) # [bs x n channels x n patches x patch len]
        x = self.flatten(x)
        x = self.conv1d(x)
        if return_softmax:
            x = self.softmax(x)
        return x

In [ ]:
#| export
class LinearProbingHead(nn.Module):
    """
    A linear probing head (with optional MLP), assumes that the d_model corresponds to a particular segment of time and will make a prediction per patch per channel, and average the results
    """
    def __init__(self, 
                 c_in, # the number of input channels in the original input
                 predict_every_n_patches, # for a given sequence of length m with frequency f, number of predictions
                 n_classes, # the number of classes to predict (for sleep stage - there are 6)
                 input_size, # the dimension of the transformer model
                 n_layers, # the number of linear layers to use in the prediciton head, with RELU activation and dropout
                 num_patch,
                 shared_embedding=True, # whether or not to have a dense layer per channel or one layer per channel
                 affine=True, # include learnable parameters to weight predictions
                 norm='BatchNorm', # batchnorm or layernorm between linear and convolutional layers
                 act='gelu', # activation function to use between layers, 'gelu' or 'relu'
                 dropout=0. # dropout in between linear layers
                 ):
        super().__init__()
        self.c_in = c_in
        self.n_classes = n_classes
        self.input_size = input_size
        self.n_layers = n_layers
        self.affine = affine
        self.shared_embedding = shared_embedding
        self.predict_every_n_patches = predict_every_n_patches
        self.num_patch = num_patch
        if self.affine:
            self.affine_weight = nn.Parameter(torch.ones(1,c_in,1))
            self.affine_bias = nn.Parameter(torch.zeros(1,c_in,1))
        
        n_nuerons = int((self.n_classes - self.input_size)/self.n_layers) if self.n_layers >= 1 else 0

        self.layers = []
        if self.n_layers == 1:
            if not shared_embedding:
                for _ in range(self.c_in): self.layers.append(nn.Linear(input_size, n_classes))
            else:
                self.layers.append(nn.Linear(input_size, n_classes)) # [bs x c_in x n_patches x n_classes]
        else:
            if not shared_embedding:
                for _ in range(self.c_in):
                    channel_layers = []
                    for i in range(self.n_layers-1):
                        channel_layers += [nn.Linear(n_nuerons*i+input_size, input_size+n_nuerons*(i+1)), nn.BatchNorm1d(self.num_patch) if "batch" in norm.lower() else nn.LayerNorm(1), get_activation_fn(act), nn.Dropout(dropout)]
                    channel_layers += [nn.Linear(input_size+n_nuerons*(i+1), self.n_classes)]
                    self.layers.append(nn.Sequential(*channel_layers))
            else:
                for i in range(self.n_layers-1):
                    self.layers.append(nn.Sequential(nn.Linear(n_nuerons*i+input_size, input_size+n_nuerons*(i+1)), Transpose(1,2), nn.BatchNorm2d(c_in) if "batch" in norm.lower() else nn.LayerNorm(c_in), Transpose(1,2), get_activation_fn(act), nn.Dropout(dropout)))
                self.layers.append(nn.Linear(input_size+n_nuerons*(i+1), self.n_classes))
        self.layers = nn.Sequential(*self.layers) # [bs x c_in x n_patches x n_classes], need to average or max over channels
        self.flatten = nn.Flatten(start_dim=-3, end_dim=-2)
        self.avg_pool_kernel_stride = int(self.c_in*self.predict_every_n_patches) if self.predict_every_n_patches > 1 else self.c_in
        self.avg_pool = nn.AvgPool1d(kernel_size=self.avg_pool_kernel_stride, stride=self.avg_pool_kernel_stride)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x, return_softmax=False):
        """
        Make sure to freeze the encoder and remove its pretrain head! 

        in: [bs x n channels x d model x n patches]
        out: [bs x n classes x pred len]
        """
        x = x.permute(0,3,1,2) # [bs x n patches x n channels x d_model]
        if not self.shared_embedding:
            x_out = []
            for i in range(self.c_in): 
                z = self.layers[i](x[:,:,i,:])
                x_out.append(z)
            x = torch.stack(x_out, dim=2) # [bs x n patches x n channels x n_classes]
        else:
            x = self.layers(x) # [bs x n patches x n channels x n_classes]
        if self.affine:
            x = x * self.affine_weight
            x = x + self.affine_bias
        x = self.flatten(x)
        x = x.transpose(1,2)
        x  = self.avg_pool(x)
        if return_softmax:
            x = self.softmax(x)
        return x

In [ ]:
#| notest
m = LinearProbingHead(c_in=7, 
                      input_size = 512, 
                      predict_every_n_patches=5,
                      n_classes=5,
                      n_layers=3,
                      shared_embedding=True,
                      affine=True,
                      num_patch=3600,
                      dropout=0.1)

x = torch.randn((4,7,512,3600))

m(x, return_softmax=True).shape

torch.Size([4, 5, 720])

In [ ]:
#| export
class TimeDistributedFeedForward(nn.Module):
    """
    Feed forward head that uses a convolutional layer to reduce channel dimensionality
    Followed by a feedforward network to make 
    """
    def __init__(self, 
                 c_in, # the number of input channels
                 n_classes, # the number of classes to predict (for sleep stage - there are 6)
                 n_patches, # the number of stft or time patches
                 d_model, # the dimension of the transformer model
                 pred_len_seconds, # the sequence multiclass prediction length in seconds
                 n_linear_layers, # the number of linear layers to use in the prediciton head, with RELU activation and dropout
                 conv_kernel_stride_size, # the 1d convolution kernel size and stride, in seconds. If you want every 30 second predicitons, put 30 here.
                 dropout=0. # dropout in between linear layers
                 ):
        super().__init__()
        self.c_in = c_in
        self.n_classes = n_classes
        self.pred_len_seconds = pred_len_seconds
        self.n_linear_layers = n_linear_layers
        self.d_model = d_model
        self.n_patches = n_patches
        self.pool_stride = (d_model*n_patches)//pred_len_seconds
        self.pool_kernel = (d_model*n_patches) - (pred_len_seconds - 1) * self.pool_stride
        #self.pred_len = int(self.pred_len_seconds / (self.hop_length / self.frequency))  # lets predict on a second basis, not on a timepoint basis
        self.flatten = nn.Flatten(start_dim=-2)
        self.layers = []
        n_nuerons = int((self.pred_len_seconds - d_model*n_patches)/self.n_linear_layers) if self.n_linear_layers >= 1 else 0
        if self.n_linear_layers == 1:
            self.layers.append(nn.Linear(d_model*n_patches, self.pred_len_seconds))
        elif self.n_linear_layers == 0:
            self.layers.append(nn.AvgPool1d(kernel_size=self.pool_kernel, stride=self.pool_stride, padding=0))
        else:
            for i in range(self.n_linear_layers-1):
                self.layers.append(nn.Sequential(nn.Linear(n_nuerons*i+d_model*n_patches, d_model*n_patches+n_nuerons*(i+1)), nn.ReLU(), nn.Dropout(dropout)))
            self.layers.append(nn.Linear(d_model*n_patches + n_nuerons*(i+1), self.pred_len_seconds))
        self.layers = nn.Sequential(*self.layers)
        self.conv1d = nn.Conv1d(c_in, n_classes, kernel_size=conv_kernel_stride_size, stride=conv_kernel_stride_size)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x, return_softmax=False):
        """
        Make sure to freeze the encoder and remove its pretrain head! 

        in: [bs x n channels x d model x n patches]
        out: [bs x n classes x pred len]
        """
        x = x.transpose(2,3)
        x = self.flatten(x)
        x = self.layers(x)
        x = self.conv1d(x)
        if return_softmax:
            x = self.softmax(x)
        return x

In [ ]:
#| export
class ConvGRU1DCell(nn.Module):
    """
    Generate a convolutional GRU cell
    """

    def __init__(self, input_size, hidden_size, kernel_size):
        super().__init__()
        padding = kernel_size // 2
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.reset_gate = nn.Conv2d(input_size + hidden_size, hidden_size, kernel_size, padding=padding)
        self.update_gate = nn.Conv2d(input_size + hidden_size, hidden_size, kernel_size, padding=padding)
        self.out_gate = nn.Conv2d(input_size + hidden_size, hidden_size, kernel_size, padding=padding)

        init.orthogonal_(self.reset_gate.weight)
        init.orthogonal_(self.update_gate.weight)
        init.orthogonal_(self.out_gate.weight)
        init.zeros_(self.reset_gate.bias)
        init.zeros_(self.update_gate.bias)
        init.zeros_(self.out_gate.bias)


    def forward(self, input_, prev_state):

        # get batch and spatial sizes
        batch_size = input_.data.size()[0]
        spatial_size = input_.data.size()[2:]

        # generate empty prev_state, if None is provided
        if prev_state is None:
            state_size = [batch_size, self.hidden_size] + list(spatial_size)
            prev_state = torch.zeros(state_size, device=input_.device)
            
        # data size is [batch, channel, height, width]
        stacked_inputs = torch.cat([input_, prev_state], dim=1)
        update = F.sigmoid(self.update_gate(stacked_inputs))
        reset = F.sigmoid(self.reset_gate(stacked_inputs))
        out_inputs = F.tanh(self.out_gate(torch.cat([input_, prev_state * reset], dim=1)))
        new_state = prev_state * (1 - update) + out_inputs * update

        return new_state


class ConvGRU1D(nn.Module):
    def __init__(self, input_size, hidden_sizes, kernel_sizes, n_layers):
        '''
        Generates a multi-layer convolutional GRU.
        Preserves spatial dimensions across cells, only altering depth.

        Parameters
        ----------
        input_size : integer. depth dimension of input tensors.
        hidden_sizes : integer or list. depth dimensions of hidden state.
            if integer, the same hidden size is used for all cells.
        kernel_sizes : integer or list. sizes of Conv2d gate kernels.
            if integer, the same kernel size is used for all cells.
        n_layers : integer. number of chained `ConvGRUCell`.
        '''

        super(ConvGRU1D, self).__init__()

        self.input_size = input_size
        if type(hidden_sizes) != list:
            self.hidden_sizes = [hidden_sizes]*n_layers
        else:
            assert len(hidden_sizes) == n_layers, '`hidden_sizes` must have the same length as n_layers'
            self.hidden_sizes = hidden_sizes
        if type(kernel_sizes) != list:
            self.kernel_sizes = [kernel_sizes]*n_layers
        else:
            assert len(kernel_sizes) == n_layers, '`kernel_sizes` must have the same length as n_layers'
            self.kernel_sizes = kernel_sizes

        self.n_layers = n_layers
        cells = nn.ModuleList()
        for i in range(self.n_layers):
            if i == 0:
                input_dim = self.input_size
            else:
                input_dim = self.hidden_sizes[i-1]

            cell = ConvGRU1DCell(input_dim, self.hidden_sizes[i], self.kernel_sizes[i])
            name = 'ConvGRU1DCell_' + str(i).zfill(2)

            setattr(self, name, cell)
            cells.append(getattr(self, name))
        self.cells = cells

    def forward(self, x, hidden=None):
        '''
        x: [batch, n_channels, dim_model, n_patches]
        hidden: [batch, n_channels, dim_model, n_patches]

        Returns
        -------
        upd_cell_hidden: [batch, n_channels, dim_model, n_patches]
        '''
        if not hidden:
            hidden = [None]*self.n_layers

        input_ = x

        upd_hidden = []

        for layer_idx in range(self.n_layers):
            cell = self.cells[layer_idx]
            cell_hidden = hidden[layer_idx]

            # pass through layer
            upd_cell_hidden = cell(input_, cell_hidden)
            #upd_hidden.append(upd_cell_hidden)
            # update input_ to the last updated hidden layer for next pass
            input_ = upd_cell_hidden

        # retain tensors in list to allow different hidden sizes
        return upd_cell_hidden

class ConvBiGRU(nn.Module):
    # Constructor
    def __init__(self, input_size, hidden_sizes, kernel_sizes, n_layers, d_model, predict_every_n_patches, n_classes):
        super(ConvBiGRU, self).__init__()
        self.flatten = nn.Flatten(start_dim=-2, end_dim=-1)

        self.forward_net = ConvGRU1D(input_size, hidden_sizes, kernel_sizes, n_layers)
        self.reverse_net = ConvGRU1D(input_size, hidden_sizes, kernel_sizes, n_layers)

        final_kernel_stride = int(d_model*predict_every_n_patches) # this will make one predictions for every predict_every_seconds seq length
        self.conv1d = nn.Conv1d(in_channels=hidden_sizes*2, out_channels=n_classes, kernel_size=final_kernel_stride, stride=final_kernel_stride)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, return_softmax=False):
        """
        x: [batch, n_channels, dim_model, n_patches]
        """
        x = x.permute(0, 1, 3, 2)
        #x = self.flatten(x)
        x_rev = torch.flip(x, dims=[-1])
        y_out_fwd = self.forward_net(x)
        y_out_rev = self.reverse_net(x_rev)
        
        y_out_rev = torch.flip(y_out_rev, dims=[-1])
        #reversed_idx = list(reversed(range(y_out_rev.shape[1])))
        #y_out_rev = y_out_rev[:, reversed_idx, ...] # reverse temporal outputs.
        ycat = torch.cat((y_out_fwd, y_out_rev), dim=1)
        ycat = self.flatten(ycat)
        ycat = self.conv1d(ycat)
        if return_softmax:
            ycat = self.softmax(ycat)
        return ycat

In [ ]:
#| notest
x = torch.randn((4,7,512,3600))

convgru = ConvBiGRU(input_size=7, hidden_sizes=32, kernel_sizes=3, n_layers=1, d_model=512, predict_every_n_patches=5, n_classes=5)

out = convgru(x)
out.shape

torch.Size([4, 5, 720])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()